In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-12-04 01:52:05--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-12-04 01:52:06--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-12-04 01:52:06 (76.3 MB/s) - written to stdout [1191/1191]

Installing

# Header and Imports

In [ ]:
import sparknlp
spark = sparknlp.start(gpu=True)

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.1.4
Apache Spark version: 3.2.3


In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from sklearn.metrics import classification_report, accuracy_score

# 3 a)No Preprocessing

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")


testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
trainDataset.show()
testDataset.show()

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business|Ilford Imaging, t...|
|Business|US Treasury Secre...|
|Business|Franklin Templeto...|
|Business|The number of mor...|
|Business|The Red Cross has...|
|Business|Based on a CDMA E...|
|Business| The U.S. House o...|
|Business|The first crimina...|
|Business|A day after a pop...|
|Business|TORONTO Press bar...|
|Business|AFTER months of p...|
|Business|The US must cut i...|
|Business|Tillie, the clown...|
|Business|India News: New D...|
|Business|Royal Group Techn...|
|Business|As they traded ch...|
|Business|Google Inc. is ex...|
|Business|Iron Mountain mov...|
|Business|Fading hopes for ...|
|Business| There's nothing ...|
+--------+--------------------+
only showing top 20 rows

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business|Unions representi...|
|Sci/Tech| TORONTO, Canada ...|
|Sci/Tech| A company founde...|
|Sci/Tech| It'

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en') \
      .setInputCols(["document","token"])\
      .setOutputCol("bert")\
      .setCaseSensitive(False)\

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings")\
      .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class") \
      .setLabelColumn("category") \
      .setMaxEpochs(5) \
      .setLr(0.001) \
      .setEnableOutputLogs(True)

bert_base_cased= Pipeline(
    stages=[document_assembler,
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classifier])

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]


In [ ]:
%%time
bert_model = bert_base_cased.fit(trainDataset)
bert_processed = bert_model.transform(trainDataset)
bert_processed.count()

CPU times: user 4.01 s, sys: 500 ms, total: 4.51 s
Wall time: 15min 55s


32000

In [ ]:
!cd ~/annotator_logs && ls -l

total 4
-rw-r--r-- 1 root root 448 Dec  4 02:12 ClassifierDLApproach_e2bbd6c7e247.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_e2bbd6c7e247.log

cat: /root/annotator_logs/ClassifierDLApproach_d56ebb1bac7a.log: No such file or directory


In [ ]:
model_df = bert_model.transform(testDataset)
model_df.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
|category|         description|    result|
+--------+--------------------+----------+
|Business|Unions representi...|[Business]|
|Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
|Sci/Tech| A company founde...|[Sci/Tech]|
|Sci/Tech| It's barely dawn...|  [Sports]|
|Sci/Tech| Southern Califor...|[Sci/Tech]|
|Sci/Tech|"The British Depa...|   [World]|
|Sci/Tech|"confessed author...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and ...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police...|   [World]|
|Sci/Tech|In its first two ...|[Business]|
+--------+--------------------+----------+
only showing top 10 rows



In [ ]:
df=model_df.select('category','description',"class.result").toPandas()
df["result"]=df["result"].apply(lambda x:x[0])

In [ ]:
print("Accuracy:", accuracy_score(df.category,df.result))
print(classification_report(df.category,df.result))

Accuracy: 0.8723684210526316
              precision    recall  f1-score   support

    Business       0.81      0.84      0.83      1900
    Sci/Tech       0.84      0.84      0.84      1900
      Sports       0.94      0.95      0.94      1900
       World       0.90      0.85      0.88      1900

    accuracy                           0.87      7600
   macro avg       0.87      0.87      0.87      7600
weighted avg       0.87      0.87      0.87      7600



# 3 b) (i) with preprocessing step -  Lemmatization

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

lemmatizer = LemmatizerModel.pretrained('lemma_antbnc', 'en') \
      .setInputCols(["normalized"]) \
      .setOutputCol("lemma")

bert_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en') \
      .setInputCols(["document","lemma"])\
      .setOutputCol("bert")\
      .setCaseSensitive(False)\

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class") \
      .setLabelColumn("category") \
      .setMaxEpochs(5) \
      .setLr(0.001) \
      .setEnableOutputLogs(True)

bert_cased_lemma = Pipeline(
    stages=[document_assembler,
            tokenizer,
            normalizer,
            lemmatizer,
            bert_embeddings,
            embeddingsSentence,
            classifier])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]


In [ ]:
%%time
bert_model_lemma = bert_cased_lemma.fit(trainDataset)
bert_processed_lemma =bert_model_lemma.transform(trainDataset)
bert_processed_lemma.count()

CPU times: user 4.64 s, sys: 602 ms, total: 5.24 s
Wall time: 15min 13s


32000

In [ ]:
!cd ~/annotator_logs && ls -l

total 8
-rw-r--r-- 1 root root 443 Dec  3 21:08 ClassifierDLApproach_d56ebb1bac7a.log
-rw-r--r-- 1 root root 446 Dec  3 21:48 ClassifierDLApproach_df0aff5db7a7.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_df0aff5db7a7.log

Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 64 - training_examples: 32000 - classes: 4
Epoch 0/5 - 1.82s - loss: 462.1406 - acc: 0.836875 - batches: 500
Epoch 1/5 - 1.22s - loss: 445.74304 - acc: 0.865875 - batches: 500
Epoch 2/5 - 1.25s - loss: 443.4002 - acc: 0.8709375 - batches: 500
Epoch 3/5 - 1.25s - loss: 442.35062 - acc: 0.8733125 - batches: 500
Epoch 4/5 - 1.23s - loss: 441.28506 - acc: 0.87453127 - batches: 500


In [ ]:
model_df = bert_model_lemma.transform(testDataset)
model_df.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
|category|         description|    result|
+--------+--------------------+----------+
|Business|Unions representi...|[Business]|
|Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
|Sci/Tech| A company founde...|[Sci/Tech]|
|Sci/Tech| It's barely dawn...|[Sci/Tech]|
|Sci/Tech| Southern Califor...|[Sci/Tech]|
|Sci/Tech|"The British Depa...|[Sci/Tech]|
|Sci/Tech|"confessed author...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and ...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police...|   [World]|
|Sci/Tech|In its first two ...|[Business]|
+--------+--------------------+----------+
only showing top 10 rows



In [ ]:
df=model_df.select('category','description',"class.result").toPandas()
df["result"]=df["result"].apply(lambda x:x[0])

In [ ]:
print("Accuracy:", accuracy_score(df.category,df.result))
print(classification_report(df.category,df.result))

Accuracy: 0.8593421052631579
              precision    recall  f1-score   support

    Business       0.84      0.78      0.81      1900
    Sci/Tech       0.80      0.86      0.83      1900
      Sports       0.91      0.96      0.93      1900
       World       0.89      0.84      0.86      1900

    accuracy                           0.86      7600
   macro avg       0.86      0.86      0.86      7600
weighted avg       0.86      0.86      0.86      7600



# 3 b) (ii) with preprocessing step -  Stop Word Removal

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")


stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("stop_cleaner")\
      .setCaseSensitive(False)

bert_embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
      .setInputCols(["document","stop_cleaner"])\
      .setOutputCol("bert")\
      .setCaseSensitive(False)\


embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class") \
      .setLabelColumn("category") \
      .setMaxEpochs(5) \
      .setLr(0.001) \
      .setEnableOutputLogs(True)

bert_base_cased_stop = Pipeline(
    stages=[document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            bert_embeddings,
            embeddingsSentence,
            classifier])

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]


In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")


testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
%%time
bert_model_stop = bert_base_cased_stop.fit(trainDataset)
bert_processed_stop =bert_model_stop.transform(trainDataset)
bert_processed_stop.count()

CPU times: user 3.32 s, sys: 420 ms, total: 3.74 s
Wall time: 10min 51s


32000

In [ ]:
!cd ~/annotator_logs && ls -l

total 12
-rw-r--r-- 1 root root 447 Dec  3 22:03 ClassifierDLApproach_c3d05269d63a.log
-rw-r--r-- 1 root root 443 Dec  3 21:08 ClassifierDLApproach_d56ebb1bac7a.log
-rw-r--r-- 1 root root 446 Dec  3 21:48 ClassifierDLApproach_df0aff5db7a7.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_df0aff5db7a7.log

Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 64 - training_examples: 32000 - classes: 4
Epoch 0/5 - 1.82s - loss: 462.1406 - acc: 0.836875 - batches: 500
Epoch 1/5 - 1.22s - loss: 445.74304 - acc: 0.865875 - batches: 500
Epoch 2/5 - 1.25s - loss: 443.4002 - acc: 0.8709375 - batches: 500
Epoch 3/5 - 1.25s - loss: 442.35062 - acc: 0.8733125 - batches: 500
Epoch 4/5 - 1.23s - loss: 441.28506 - acc: 0.87453127 - batches: 500


In [ ]:
model_df = bert_model_stop.transform(testDataset)
model_df.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
|category|         description|    result|
+--------+--------------------+----------+
|Business|Unions representi...|[Business]|
|Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
|Sci/Tech| A company founde...|[Sci/Tech]|
|Sci/Tech| It's barely dawn...|[Sci/Tech]|
|Sci/Tech| Southern Califor...|[Sci/Tech]|
|Sci/Tech|"The British Depa...|[Sci/Tech]|
|Sci/Tech|"confessed author...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and ...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police...|[Business]|
|Sci/Tech|In its first two ...|[Sci/Tech]|
+--------+--------------------+----------+
only showing top 10 rows



In [ ]:
df=model_df.select('category','description',"class.result").toPandas()
df["result"]=df["result"].apply(lambda x:x[0])

In [ ]:
print("Accuracy:", accuracy_score(df.category,df.result))
print(classification_report(df.category,df.result))

Accuracy: 0.8621052631578947
              precision    recall  f1-score   support

    Business       0.81      0.81      0.81      1900
    Sci/Tech       0.83      0.84      0.83      1900
      Sports       0.90      0.96      0.93      1900
       World       0.90      0.83      0.87      1900

    accuracy                           0.86      7600
   macro avg       0.86      0.86      0.86      7600
weighted avg       0.86      0.86      0.86      7600



# 3 b) (iii) with preprocessing step -  Lemmatization and Stop Word Removal

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")


testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols(["normalized"])\
      .setOutputCol("stop")\
      .setCaseSensitive(False)

lemmatizer = LemmatizerModel.pretrained('lemma_antbnc', 'en') \
      .setInputCols(["stop"]) \
      .setOutputCol("lemma")

bert_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en') \
      .setInputCols(["document","lemma"])\
      .setOutputCol("bert")\
      .setCaseSensitive(False)\


embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class") \
      .setLabelColumn("category") \
      .setMaxEpochs(5) \
      .setLr(0.001) \
      .setEnableOutputLogs(True)

bert_base_cased_lemma_stop = Pipeline(
    stages=[document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemmatizer,
            bert_embeddings,
            embeddingsSentence,
            classifier])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]


In [ ]:
%%time
bert_model_lemma_stop = bert_base_cased_lemma_stop.fit(trainDataset)
bert_processed_lemma_stop =bert_model_lemma_stop.transform(trainDataset)
bert_processed_lemma_stop.count()

CPU times: user 4.07 s, sys: 481 ms, total: 4.55 s
Wall time: 14min 29s


32000

In [ ]:
!cd ~/annotator_logs && ls -l

total 4
-rw-r--r-- 1 root root 447 Dec  3 22:49 ClassifierDLApproach_50cb5405584f.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_50cb5405584f.log

Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 64 - training_examples: 32000 - classes: 4
Epoch 0/5 - 1.95s - loss: 448.73026 - acc: 0.83053124 - batches: 500
Epoch 1/5 - 1.24s - loss: 434.63428 - acc: 0.85753125 - batches: 500
Epoch 2/5 - 1.36s - loss: 431.3939 - acc: 0.8629375 - batches: 500
Epoch 3/5 - 1.74s - loss: 429.86963 - acc: 0.86625 - batches: 500
Epoch 4/5 - 1.54s - loss: 429.20758 - acc: 0.868625 - batches: 500


In [ ]:
model_df = bert_model_lemma_stop.transform(testDataset)
model_df.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
|category|         description|    result|
+--------+--------------------+----------+
|Business|Unions representi...|[Business]|
|Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
|Sci/Tech| A company founde...|[Sci/Tech]|
|Sci/Tech| It's barely dawn...|[Sci/Tech]|
|Sci/Tech| Southern Califor...|[Sci/Tech]|
|Sci/Tech|"The British Depa...|[Sci/Tech]|
|Sci/Tech|"confessed author...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and ...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police...|[Business]|
|Sci/Tech|In its first two ...|[Business]|
+--------+--------------------+----------+
only showing top 10 rows



In [ ]:
df=model_df.select('category','description',"class.result").toPandas()
df["result"]=df["result"].apply(lambda x:x[0])

In [ ]:
print("Accuracy:", accuracy_score(df.category,df.result))
print(classification_report(df.category,df.result))

Accuracy: 0.8581578947368421
              precision    recall  f1-score   support

    Business       0.81      0.81      0.81      1900
    Sci/Tech       0.84      0.81      0.82      1900
      Sports       0.91      0.95      0.93      1900
       World       0.87      0.86      0.87      1900

    accuracy                           0.86      7600
   macro avg       0.86      0.86      0.86      7600
weighted avg       0.86      0.86      0.86      7600



# 3 c) RoBerta embeddings with no preprocessing as it gave highest accuracy

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")


testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
trainDataset.show()
testDataset.show()

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business|Ilford Imaging, t...|
|Business|US Treasury Secre...|
|Business|Franklin Templeto...|
|Business|The number of mor...|
|Business|The Red Cross has...|
|Business|Based on a CDMA E...|
|Business| The U.S. House o...|
|Business|The first crimina...|
|Business|A day after a pop...|
|Business|TORONTO Press bar...|
|Business|AFTER months of p...|
|Business|The US must cut i...|
|Business|Tillie, the clown...|
|Business|India News: New D...|
|Business|Royal Group Techn...|
|Business|As they traded ch...|
|Business|Google Inc. is ex...|
|Business|Iron Mountain mov...|
|Business|Fading hopes for ...|
|Business| There's nothing ...|
+--------+--------------------+
only showing top 20 rows

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business|Unions representi...|
|Sci/Tech| TORONTO, Canada ...|
|Sci/Tech| A company founde...|
|Sci/Tech| It'

In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 7.6 MB/s eta 0:00:00


In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

roberta_embeddings = RoBertaEmbeddings.pretrained('roberta_base', 'en') \
      .setInputCols(["document",'token'])\
      .setOutputCol("roberta")\
      .setCaseSensitive(False)\

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "roberta"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class") \
      .setLabelColumn("category") \
      .setMaxEpochs(5) \
      .setLr(0.001) \
      .setEnableOutputLogs(True)

roberta_base_cased = Pipeline(
    stages=[document_assembler,
            tokenizer,
            roberta_embeddings,
            embeddingsSentence,
            classifier])

roberta_base download started this may take some time.
Approximate size to download 284.3 MB
[OK!]


In [ ]:
%%time
roberta_model = roberta_base_cased.fit(trainDataset)
roberta_processed = roberta_model.transform(trainDataset)
roberta_processed.count()

CPU times: user 6.13 s, sys: 733 ms, total: 6.87 s
Wall time: 24min 19s


32000

In [ ]:
!cd ~/annotator_logs && ls -l

total 4
-rw-r--r-- 1 root root 446 Dec  4 00:03 ClassifierDLApproach_e48cd66589ae.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_e48cd66589ae.log

cat: /root/annotator_logs/ClassifierDLApproach_e48cd66589ae.log: No such file or directory


In [ ]:
model_df = roberta_model.transform(testDataset)
model_df.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
|category|         description|    result|
+--------+--------------------+----------+
|Business|Unions representi...|[Sci/Tech]|
|Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
|Sci/Tech| A company founde...|[Sci/Tech]|
|Sci/Tech| It's barely dawn...|  [Sports]|
|Sci/Tech| Southern Califor...|   [World]|
|Sci/Tech|"The British Depa...|[Sci/Tech]|
|Sci/Tech|"confessed author...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and ...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police...|   [World]|
|Sci/Tech|In its first two ...|[Business]|
+--------+--------------------+----------+
only showing top 10 rows



In [ ]:
df=model_df.select('category','description',"class.result").toPandas()
df["result"]=df["result"].apply(lambda x:x[0])

In [ ]:
print("Accuracy:", accuracy_score(df.category,df.result))
print(classification_report(df.category,df.result))

Accuracy: 0.7985526315789474
              precision    recall  f1-score   support

    Business       0.76      0.77      0.76      1900
    Sci/Tech       0.76      0.75      0.75      1900
      Sports       0.88      0.89      0.88      1900
       World       0.79      0.79      0.79      1900

    accuracy                           0.80      7600
   macro avg       0.80      0.80      0.80      7600
weighted avg       0.80      0.80      0.80      7600

